<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/concurrency_and_asynchornous_dependency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complex Process using AsyncIO to Manage Concurrency and Asynchronous Dependencies

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** None

**Description:**

Async is very important when we try to manage a complex process. Agently framework provides `.start_async()` to allow developers create their own async processes.

Take this process down below as an example, let see how to use asyncio and Agently to manage concurrency and asynchronous dependencies.

当我们试图管理一个复杂流程的时候，对异步（async）的支持非常重要。Agently框架提供`.start_async()`的方式来允许开发者进行异步请求。

让我们通过案例来看看如何使用asyncio和Agently来实现如下所示的流程，处理并发和异步依赖。

```
[generate 3 colors] -> [generate 3 sentences using 3 colors] ->|
                                                               |-> [tell a story using 3 sentences and 3 names]
[generate 2 boys' names and 1 girl's name] --------------------|
```

## Step 1: Install Packages

In [ ]:
!pip install -U Agently
!pip install nest_asyncio

## Step 2: Demo Code

In [4]:
import time
import asyncio
import Agently

agent_factory = (
    Agently.AgentFactory()
        .set_settings("model.OpenAI.auth", { "api_key": "" })
)

agent_a = agent_factory.create_agent()
agent_b = agent_factory.create_agent()

async def generate_sentences():
    start_time = time.time()
    print(f"[Generate Sentences] Start at: { start_time }")
    job1_time = time.time()
    print(f"[Generate Sentences] Job 1 start at: { job1_time }")
    colors = (
        await agent_a
            .input("Generate 3 colors.")
            .output([("String", "color")])
            .start_async()
    )
    print("[Generate Sentences] Colors", colors)
    job2_time = time.time()
    print(f"[Generate Sentences] Job 2 start at: { job2_time }")
    sentences = (
        await agent_a
            .input(colors)
            .instruct("Generate 3 sentences using different color in {input}")
            .output([("String", "sentence")])
            .start_async()
    )
    print("[Generate Sentences]")
    for sentence in sentences:
        print(sentence)
    end_time = time.time()
    print(f"[Generate Sentences] Finish at: { end_time }")
    print(f"[Generate Sentences] Time cost: { end_time - start_time }")
    return sentences

async def generate_names():
    start_time = time.time()
    print(f"[Generate Names] Start at: { start_time }")
    names = (
        await agent_b
            .input("Generate 2 boys' name and 1 girl's name")
            .output({
                "boy_names": [("String", )],
                "girl_name": [("String", )],
            })
            .start_async()
    )
    print("[Generate Names]")
    print(names)
    end_time = time.time()
    print(f"[Generate Names] Finish at: { end_time }")
    print(f"[Generate Names] Time cost: { end_time - start_time }")
    return names

async def tell_story(sentences, names):
    start_time = time.time()
    print(f"[Tell Story] Start at: { start_time }")
    story = (
        await agent_a
            .input({
                "sentences": sentences,
                "names": names,
            })
            .instruct(
                "Tell a story that characters using names in {names}" +
                " and must include sentences in {sentences}"
            )
            .start_async()
    )
    print("[Tell Story]")
    print(story)
    end_time = time.time()
    print(f"[Tell Story] Finish at: { end_time }")
    print(f"[Tell Story] Time cost: { end_time - start_time }")
    return story

async def main():
    sentences, names = await asyncio.gather(
        generate_sentences(),
        generate_names()
    )
    await tell_story(sentences, names)

# use nest_asyncio to enable asyncio.run() in colab
import nest_asyncio
nest_asyncio.apply()

asyncio.run(main())

[Generate Sentences] Start at: 1703606150.0052722
[Generate Sentences] Job 1 start at: 1703606150.0053837
[Generate Names] Start at: 1703606150.0946443
[Generate Sentences] Colors ['red', 'blue', 'green']
[Generate Sentences] Job 2 start at: 1703606151.4804964
[Generate Names]
{'boy_names': ['James', 'Oliver'], 'girl_name': ['Emma']}
[Generate Names] Finish at: 1703606152.2673044
[Generate Names] Time cost: 2.1726601123809814
[Generate Sentences]
The red apple is juicy.
The sky is blue.
The leaves on the tree are green.
[Generate Sentences] Finish at: 1703606153.2162726
[Generate Sentences] Time cost: 3.211000442504883
[Tell Story] Start at: 1703606153.2164001
[Tell Story]
Once upon a time, in a small town called Appleville, lived three friends named James, Oliver, and Emma. James was known for his love for nature, Oliver was a curious and adventurous soul, and Emma was the kindest and most intelligent girl in town.

One sunny afternoon, James, Oliver, and Emma decided to go on a natur

#### How to Read Output Logs

First of all, we require async functions [Generate Sentences] and [Generate Names] to start at the same time:

首先，根据流程图示，我们需要[Generate Sentences]和[Generate Names]两个执行函数同时被执行，在输出中可以看到：

```
[Generate Sentences] Start at: 1703606150.0052722
[Generate Names] Start at: 1703606150.0946443
```
OK, those two functions started at almost the same time.

说明两个函数启动执行的时间相差不远。

Then we require two process 'generate colors' and 'generate sentences' proceed in orders:

然后，我们需要[Generate Sentences]内部的两个步骤顺序执行，在输出中可以看到：

```
[Generate Sentences] Job 1 start at: 1703606150.0053837
[Generate Sentences] Job 2 start at: 1703606151.4804964
[Generate Sentences] Finish at: 1703606153.2162726
```

Also we can check the colors result and the sentences result in the output logs. Sentences result indeed included colors generated at the first time.

同时，我们也可以从输出中看到颜色(colors)和句子(sentences)的结果，句子的确使用了之前生成的颜色内容，说明执行顺序没有问题。

```
[Generate Sentences] Colors ['red', 'blue', 'green']
[Generate Sentences]
The red apple is juicy.
The sky is blue.
The leaves on the tree are green.
```

And we can see [Generate Names] finished in the middle of the [Generate Sentences] process:

我们也可以看到[Generate Names]在[Generate Sentences]的执行过程中完成了自己的执行：

```
[Generate Names]
{'boy_names': ['James', 'Oliver'], 'girl_name': ['Emma']}
[Generate Names] Finish at: 1703606152.2673044
```

At last, [Tell Story] started when all other processes were finished:

最后，[Tell Story]这个函数是在所有其他进程都执行完毕后，才开始执行：

```
[Tell Story] Start at: 1703606153.2164001
```

The output logs shows all functions executed in the orders we required.

与我们预期的执行顺序一致。

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)